In [1]:
import math
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
from sklearn.utils import shuffle
import nltk
import gensim
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

2023-07-14 15:16:21.555269: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-14 15:16:22.410441: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-14 15:16:24.052333: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-14 15:16:24.052782: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

### Building the Vocabulary

First, we build the vocabulary dictionaries for the source and target. 
The vocabulary is the the file `vocab.txt` (generated in the other script).

In [2]:
# Word string -> ID mapping
dictionary = dict()

with open('data/vocab.30K.txt', encoding='utf-8') as f:
    for line in f:
        # disregard the new line aka `\n`
        dictionary[line[:-1]] = len(dictionary)
        
reverse_dictionary = dict(zip(dictionary.values(),dictionary.keys()))

print('Dictionary:', list(dictionary.items())[:10], end = '\n')
print('Reverse dictionary:', list(reverse_dictionary.items())[:10], end = '\n')
print('Vocabulary size: ', len(dictionary), end = '\n')


Dictionary: [('<unk>', 0), ('<s>', 1), ('</s>', 2), ('.', 3), ('the', 4), (',', 5), ('a', 6), ('?', 7), ('to', 8), ('you', 9)]
Reverse dictionary: [(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, '.'), (4, 'the'), (5, ','), (6, 'a'), (7, '?'), (8, 'to'), (9, 'you')]
Vocabulary size:  30000


### Loading the Dataset
Here we load the data from the `dataset.csv` file (generated in the other script)

In [6]:
dataset = pd.read_csv('data/dataset.csv')

### Data pre-processing
Transform to lower, remove the new line and the punctuation

In [7]:
def lowerDataset(data):
    return data.str.lower() 
    
def cleanDataset(data):
    return data.str.replace(r"[^a-zA-Z0-9?'.,]+",' ',regex=True)
    
def paddDataset(data):
    return data.str.replace(',', ' ,')                 \
                .str.replace('.',' . ', regex=False)    \
                .str.replace('?',' ?', regex=False)    \
                .str.replace('\n',' ')


In [8]:
wt = nltk.tokenize.WhitespaceTokenizer()
for column in dataset.columns:    
    dataset[column] = lowerDataset(dataset[column]) 
    dataset[column] = cleanDataset(dataset[column])
    dataset[column] = paddDataset(dataset[column])                                    
    dataset[column] = dataset[column].apply(wt.tokenize)
dataset = shuffle(dataset)

In [9]:
dataset.head()

,question,answer
232316,"what is something that frustrates you , but to...",okay this is my time to shine . been thinking...
194487,i have cancer and will start chemo next month ...,hi good luck with everything . i'm a cancer ...
222270,what first world problem are you currently dea...,reddit is distracting me from my tv show
446293,my grandfather shot and killed himself yesterd...,"first off , i'm terribly sorry for your loss ..."
890004,"reddit people , what made you smile laugh today ?",my girlfriend farted in bed


### Data analysis
Mean sentence length and standard deviation of sentence length

In [7]:
print('Central tendency, dispersion and shape of questions’s distribution')
print(dataset['question'].str.len().describe().apply(lambda x: format(x, 'f')))

Central tendency, dispersion and shape of questions’s distribution
count    1149819.000000
mean          17.113906
std            9.139078
min            1.000000
25%           11.000000
50%           15.000000
75%           21.000000
max           82.000000
Name: question, dtype: object


In [8]:
print('Central tendency, dispersion and shape of answers’s distribution')
print(dataset['answer'].str.len().describe().apply(lambda x: format(x, 'f')))

Central tendency, dispersion and shape of answers’s distribution
count    1149819.000000
mean          54.452828
std          844.371854
min            0.000000
25%           10.000000
50%           22.000000
75%           53.000000
max       563680.000000
Name: answer, dtype: object


### Update the sentences to fixed length
Update all sentences with a fixed size, to process the sentences as batches.

In [9]:
max_sent_length = {'question' : 30, 'answer': 60}

def padding_sent(source):
    padded = []
    for tokens in dataset[source]: 
        # adding the start token
        tokens.insert(0, '<s>')  

        if len(tokens) >= max_sent_length[source]:
            tokens = tokens[:(max_sent_length[source] - 1)]
            tokens.append('</s>')

        if len(tokens) < max_sent_length[source]:
            tokens.extend(['</s>' for _ in range(max_sent_length[source] - len(tokens))])  

        padded.append(tokens)
    return padded

questions = padding_sent('question')
answers = padding_sent('answer')

### Create the reverse dataset
The reverse dataset are going to be used to retrieve the decoder output to text

In [10]:
def create_reverse_dataset(source):
    reverse_tokens = []
    reverse_dataset = []
    for tokens in source: 
        for token in tokens: 
            if token not in dictionary.keys():
                reverse_tokens.append(dictionary['<unk>'])
            else:
                reverse_tokens.append(dictionary[token])
        reverse_dataset.append(reverse_tokens)
        reverse_tokens = []
    return reverse_dataset

inputs_indexes =  np.array(create_reverse_dataset(questions), dtype=np.int32)
outputs_indexes =  np.array(create_reverse_dataset(answers), dtype=np.int32)

### Word Embedding
Use the Word2Vec to embed the input to a hight demention vector, that will keep the word relationships 

In [11]:
"""
Learns all the Word2Vec relationships fo the questions, answers and unkown words
This code needs to be run just once
Uncomment the lines to create the Word2Vec model
"""

#model = Word2Vec(questions + answers + [['<unk>']], vector_size=100, window=5, min_count=1, workers=4)
#model.save("word2vec.model")

'\nLearns all the Word2Vec relationships fo the questions, answers and unkown words\nThis code needs to be run just once\nUncomment the lines to create the Word2Vec model\n'

In [12]:
model = Word2Vec.load("word2vec.model")

### Formating the inputs and outputs

The dataset original format for inputs is 30x100 and for the outputs is 60x30000 for 1.149 million records, making the memory usage impracticable.

The inputs and outputs are goint to be refactor to 15x100 for the inputs and 30x30000 for the outputs to a total of 2.299 million records.

In [13]:
"""

input_window_size = 15
output_window_size = 30

"""

#Factor to reshape the dataset
reshape_factor = 2 

input_window_size = int(max_sent_length['question'] / reshape_factor)
output_window_size = int(max_sent_length['answer'] / reshape_factor)

def array_numpy(array):
    return np.array(array, dtype=np.float32)

def get_batch_inputs(batch, batch_size = 10):
    train_inputs = list()
    
    batch_start = batch * batch_size
    batch_end = batch_start + batch_size
    
    for input_index in inputs_indexes[batch_start:batch_end]:
        train_input = list()   
        
        for index in input_index:
            # Formates the input to the word2vec encoded format
            train_input.append(model.wv[reverse_dictionary[index]])
            
        train_inputs.append(array_numpy(train_input[:input_window_size]))
        train_inputs.append(array_numpy(train_input[input_window_size:]))
    return array_numpy(train_inputs)

def get_batch_outputs(batch, batch_size = 10):
    train_outputs = list()
    train_targets = list()
    
    batch_start = batch * batch_size
    batch_end = batch_start + batch_size
    
    for output_index in outputs_indexes[batch_start:batch_end]:
        train_output = list()
        train_target = list()
                
        for timestep, index in enumerate(output_index):
            # Formates the output to the one-hot-encode format
            output_encoded = np.zeros(len(dictionary), dtype=np.float32)
            output_encoded[index] = 1
            train_output.append(output_encoded)
            
            # Formates the target to the one-hot-encode format
            # Setted as index - 1 because it ignores the first <s>
            if timestep > 0:
                target_encoded = np.zeros(len(dictionary), dtype=np.float32)
                target_encoded[output_index[timestep]] = 1.0
                train_target.append(target_encoded)
        
        train_outputs.append(array_numpy(train_output[:output_window_size]))
        train_outputs.append(array_numpy(train_output[output_window_size:]))
        
        #Add a </s> in the end of the target so len(output) == len(taget) 
        target_encoded = np.zeros(len(dictionary), dtype=np.float32)
        target_encoded[output_index[-1]] = 1.0
        train_target.append(target_encoded)
                
        train_targets.append(array_numpy(train_target[:output_window_size]))
        train_targets.append(array_numpy(train_target[output_window_size:]))
        
    return array_numpy(train_outputs), array_numpy(train_targets)

In [14]:
input_example = get_batch_inputs(0, 1)
output_example, target_example = get_batch_outputs(0, 1)

In [15]:
print("Input shape:", input_example.shape)
print("Output shape:", output_example.shape)
print("Target shape:", target_example.shape)

Input shape: (2, 15, 100)
Output shape: (2, 30, 20000)
Target shape: (2, 30, 20000)


In [16]:
# Input shape = 100
input_shape = input_example.shape[2]

# Output shape = 30000
output_shape = output_example.shape[2]

#Dimensionality
dimensionality = 256

#The batch size and number of epochs
batch_size = 10
epochs = 150

#Encoder
encoder_inputs = Input(shape=(None, input_shape))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

#Decoder
decoder_inputs = Input(shape=(None, output_shape))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(output_shape, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Compiling
training_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')

training_model.summary()


2023-07-12 19:33:03.102530: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-12 19:33:03.129503: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-12 19:33:03.129796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-12 19:33:03.130326: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 100)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 20000  0           []                               
                                )]                                                                
                                                                                                  
 lstm (LSTM)                    [(None, 256),        365568      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                 

### Training in batches

Due to memory usage, the training will be divided into batches

In [17]:
num_records = len(dataset) * reshape_factor
data_batch_size = 500
supported_batch_records = num_records / data_batch_size
batch_records = math.ceil(supported_batch_records)

In [18]:
print("Using ", batch_records, " batches")
print("With ", data_batch_size, " (inputs, outputs) each")
print("With the totall of ", num_records, " pairs to process")

Using  91986  batches
With  25  (inputs, outputs) each
With the totall of  2299638  pairs to process


### Cleaning memory

Removing variables that are no longer needed before start the training

In [19]:
del dataset
del questions
del answers

### Start the training

In [ ]:
# Run for the batches 
for batch_record in range(batch_records + 1):
    
    # Validate for the limit size of the dataset
    # Limit the last step not to exceed the data size
    if batch_record > (num_records / data_batch_size):
        input_batch_size = math.floor((supported_batch_records % 1) * data_batch_size)
    
    # Get the inputs
    inputs_data = get_batch_inputs(batch_record, data_batch_size)
    
    # Get the outputs
    outputs_data, target_data = get_batch_outputs(batch_record, data_batch_size)
    
    #Training
    training_model.fit([inputs_data, outputs_data], outputs_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
    training_model.save('training_model.h5')
   
